In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

file_pd = 'sample_submission.csv'
pd_df = pd.read_csv(file_pd)


pd_df.head()

,row_id,article_id,dataset_id,type
0,0,10.1002_2017jc013030,https://doi.org/10.17882/49388,Primary
1,1,10.1002_cssc.202201821,https://doi.org/10.5281/zenodo.7074790,Primary
2,2,10.1002_ece3.4466,https://doi.org/10.5061/dryad.r6nq870,Primary
3,3,10.1002_ece3.5260,https://doi.org/10.5061/dryad.2f62927,Primary
4,4,10.1002_ece3.6144,https://doi.org/10.5061/dryad.zw3r22854,Primary


In [5]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

def extract_text_from_element(elem, skip_tags=("xref", "table-wrap", "fig", "table", "label")):
    """Ekstrak teks dari elemen XML, abaikan tag tertentu"""
    texts = []
    for sub in elem.iter():
        if sub.tag not in skip_tags:
            if sub.text:
                texts.append(sub.text.strip())
            if sub.tail:
                texts.append(sub.tail.strip())
    return " ".join(texts)

def parse_article_xml_clean(xml_path):
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Ambil article_id dari DOI
        doi_elem = root.find(".//article-id[@pub-id-type='doi']")
        article_id = doi_elem.text.strip().replace("/", "_") if doi_elem is not None else os.path.basename(xml_path).replace(".xml", "")

        # Ambil abstract
        abstract_elem = root.find(".//abstract")
        abstract = extract_text_from_element(abstract_elem) if abstract_elem is not None else ""

        # Ambil body (tanpa xref, figure, table)
        body_elem = root.find(".//body")
        sections = []
        if body_elem is not None:
            for sec in body_elem.findall(".//sec"):
                sec_text = extract_text_from_element(sec)
                if sec_text:
                    sections.append(sec_text)
        body = "\n\n".join(sections)

        return {
            "article_id": article_id,
            "abstract": abstract,
            "body": body,
        }

    except Exception as e:
        print(f"Parsing error in {xml_path}: {e}")
        return None

# 🔧 Sesuaikan path ini jika perlu
xml_folder = "train/XML"

# Proses semua file .xml
parsed_articles = []
for fname in os.listdir(xml_folder):
    if fname.endswith(".xml"):
        full_path = os.path.join(xml_folder, fname)
        article_data = parse_article_xml_clean(full_path)
        if article_data:
            parsed_articles.append(article_data)

# Simpan ke CSV yang rapi
df_xml = pd.DataFrame(parsed_articles)
df_xml.to_csv("parsed_articles_train_clean.csv", index=False)
print("✅ Parsing selesai. Data disimpan ke parsed_articles_train_clean.csv")


✅ Parsing selesai. Data disimpan ke parsed_articles_train_clean.csv


In [10]:
import pandas as pd

# 1. Load hasil parsing artikel
df_articles = pd.read_csv("parsed_articles_train_clean.csv")

# 2. Load label (relasi article_id ke dataset_id dan type)
df_labels = pd.read_csv("train_labels.csv")

# 3. Gabungkan ke satu DataFrame berdasarkan article_id
df_merged = df_labels.merge(df_articles, on="article_id", how="left")

# 4. Drop kolom abstract, body, dan text jika ada
df_merged = df_merged.drop(columns=["abstract", "body"], errors="ignore")
if "text" in df_merged.columns:
    df_merged = df_merged.drop(columns=["text"])

# 5. Simpan ke file CSV baru
df_merged.to_csv("train_merged_no_text.csv", index=False)

print("✅ Selesai! Semua teks dihapus. File: train_merged_no_text.csv")


✅ Selesai! Semua teks dihapus. File: train_merged_no_text.csv


In [11]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

def extract_text_from_element(elem, skip_tags=("xref", "table-wrap", "fig", "table", "label")):
    """Ekstrak teks dari elemen XML, abaikan tag tertentu"""
    texts = []
    for sub in elem.iter():
        if sub.tag not in skip_tags:
            if sub.text:
                texts.append(sub.text.strip())
            if sub.tail:
                texts.append(sub.tail.strip())
    return " ".join(texts)

def parse_article_xml_clean(xml_path):
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Ambil article_id dari DOI
        doi_elem = root.find(".//article-id[@pub-id-type='doi']")
        article_id = doi_elem.text.strip().replace("/", "_") if doi_elem is not None else os.path.basename(xml_path).replace(".xml", "")

        # Ambil abstract
        abstract_elem = root.find(".//abstract")
        abstract = extract_text_from_element(abstract_elem) if abstract_elem is not None else ""

        # Ambil body (tanpa xref, figure, table)
        body_elem = root.find(".//body")
        sections = []
        if body_elem is not None:
            for sec in body_elem.findall(".//sec"):
                sec_text = extract_text_from_element(sec)
                if sec_text:
                    sections.append(sec_text)
        body = "\n\n".join(sections)

        return {
            "article_id": article_id,
            "abstract": abstract,
            "body": body,
        }

    except Exception as e:
        print(f"Parsing error in {xml_path}: {e}")
        return None

# 🔧 Sesuaikan path ini jika perlu
xml_folder = "test/XML"

# Proses semua file .xml
parsed_articles = []
for fname in os.listdir(xml_folder):
    if fname.endswith(".xml"):
        full_path = os.path.join(xml_folder, fname)
        article_data = parse_article_xml_clean(full_path)
        if article_data:
            parsed_articles.append(article_data)

# Simpan ke CSV yang rapi
df_xml = pd.DataFrame(parsed_articles)
df_xml.to_csv("parsed_articles_test_clean.csv", index=False)
print("✅ Parsing selesai. Data disimpan ke parsed_articles_test_clean.csv")


✅ Parsing selesai. Data disimpan ke parsed_articles_test_clean.csv


In [3]:
import pandas as pd

# 1. Load hasil parsing artikel
df_articles = pd.read_csv("parsed_articles_test_clean.csv")

# 2. Load label (relasi article_id ke dataset_id dan type)
df_labels = pd.read_csv("train_merged_no_text.csv")

# 3. Gabungkan ke satu DataFrame berdasarkan article_id
df_merged = df_labels.merge(df_articles, on="article_id", how="left")

# 4. Drop kolom abstract, body, dan text jika ada
df_merged = df_merged.drop(columns=["abstract", "body"], errors="ignore")
if "text" in df_merged.columns:
    df_merged = df_merged.drop(columns=["text"])

# 5. Simpan ke file CSV baru
df_merged.to_csv("test_merged_no_text.csv", index=False)

print("✅ Selesai! Semua teks dihapus. File: test_merged_no_text.csv")


✅ Selesai! Semua teks dihapus. File: test_merged_no_text.csv


In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import joblib

# ==== 1. Load Data Train ====
df_articles = pd.read_csv("parsed_articles_train_clean.csv")  # kolom: article_id, abstract, body
df_labels = pd.read_csv("train_labels.csv")  # kolom: article_id, dataset_id, type

# Gabungkan artikel dengan label
df_train = df_labels.merge(df_articles, on="article_id", how="left")

# Gabungkan abstract + body menjadi satu kolom teks
df_train["full_text"] = (df_train["abstract"].fillna("") + " " + df_train["body"].fillna(""))

# Target dan fitur
X_text = df_train["full_text"]
y = df_train["type"]

print("Jumlah data train:", len(X_text))
print("Distribusi label:\n", y.value_counts())

# ==== 2. Split Data untuk Validasi ====
X_train, X_val, y_train, y_val = train_test_split(
    X_text, y, test_size=0.2, random_state=42, stratify=y
)

# ==== 3. Pipeline TF-IDF + Logistic Regression ====
model_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=30000, ngram_range=(1,2))),
    ('clf', LogisticRegression(max_iter=200, solver='saga'))
])

# Latih model
print("\n🔄 Training model...")
model_pipeline.fit(X_train, y_train)

# Evaluasi
y_pred = model_pipeline.predict(X_val)
print(f"✅ Akurasi validasi: {accuracy_score(y_val, y_pred):.4f}")

# ==== 4. Prediksi untuk Data Test ====
df_test = pd.read_csv("parsed_articles_test_clean.csv")  # kolom: article_id, abstract, body
df_test["full_text"] = (df_test["abstract"].fillna("") + " " + df_test["body"].fillna(""))

print("\nJumlah data test:", len(df_test))

# Prediksi label test
test_predictions = model_pipeline.predict(df_test["full_text"])

# ==== 5. Buat File Submission dengan tambahan row_id & dataset_id ====
submission = pd.DataFrame({
    "row_id": range(1, len(df_test) + 1),  # mulai dari 1
    "article_id": df_test["article_id"],
    "dataset_id": "unknown",  # karena tidak tersedia di test
    "type": test_predictions
})

# Simpan file submission
submission.to_csv("submission.csv", index=False)
print("\n✅ File submission.csv berhasil dibuat!")
print(submission.head())

# ==== 6. (Opsional) Simpan Model ====
joblib.dump(model_pipeline, "text_classification_model.pkl")
print("\n✅ Model disimpan sebagai text_classification_model.pkl")


Jumlah data train: 1028
Distribusi label:
 type
Secondary    449
Missing      309
Primary      270
Name: count, dtype: int64

🔄 Training model...
✅ Akurasi validasi: 0.6408

Jumlah data test: 25

✅ File submission.csv berhasil dibuat!
   row_id              article_id dataset_id     type
0       1    10.1002_2017jc013030    unknown  Primary
1       2  10.1002_anie.201916483    unknown  Missing
2       3  10.1002_anie.202005531    unknown  Primary
3       4  10.1002_anie.202007717    unknown  Missing
4       5  10.1002_chem.201902131    unknown  Missing

✅ Model disimpan sebagai text_classification_model.pkl


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
import keras_tuner as kt

# ==== 1. Load & Preprocess Data ====
df_articles = pd.read_csv("parsed_articles_train_clean.csv")  # abstract + body
df_labels = pd.read_csv("train_labels.csv")

# Gabungkan teks
df = df_labels.merge(df_articles, on="article_id", how="left")
df["full_text"] = (df["abstract"].fillna("") + " " + df["body"].fillna(""))

# Encode label
le = LabelEncoder()
df["label"] = le.fit_transform(df["type"])

X_text = df["full_text"].tolist()
y = to_categorical(df["label"], num_classes=len(le.classes_))

# Split data
X_train_text, X_val_text, y_train, y_val = train_test_split(X_text, y, test_size=0.2, random_state=42, stratify=y)

# Tokenizer
max_words = 30000
max_len = 500
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train_text)

X_train = pad_sequences(tokenizer.texts_to_sequences(X_train_text), maxlen=max_len)
X_val = pad_sequences(tokenizer.texts_to_sequences(X_val_text), maxlen=max_len)

# ==== 2. Define CNN Model with Hyperparameters ====
def build_model(hp):
    model = Sequential()
    model.add(Embedding(max_words, hp.Int('embedding_dim', 100, 300, step=50), input_length=max_len))
    model.add(Conv1D(filters=hp.Int('filters', 64, 256, step=64),
                     kernel_size=hp.Choice('kernel_size', [3, 5, 7]),
                     activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(hp.Float('dropout', 0.3, 0.6, step=0.1)))
    model.add(Dense(len(le.classes_), activation='softmax'))
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# ==== 3. Hyperparameter Tuning ====
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        executions_per_trial=1,
                        directory='cnn_tuning',
                        project_name='text_classification')

tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_val, y_val),
             batch_size=32)

best_hp = tuner.get_best_hyperparameters(1)[0]
print("Best Hyperparameters:")
print(f"Embedding Dim: {best_hp.get('embedding_dim')}")
print(f"Filters: {best_hp.get('filters')}")
print(f"Kernel Size: {best_hp.get('kernel_size')}")
print(f"Dropout: {best_hp.get('dropout')}")

# ==== 4. Train Final Model ====
model = tuner.hypermodel.build(best_hp)
history = model.fit(X_train, y_train,
                    epochs=10,
                    validation_data=(X_val, y_val),
                    batch_size=32)

# ==== 5. Save Model ====
model.save("cnn_text_classification.h5")

# ==== 6. Test Data Prediction ====
df_test = pd.read_csv("parsed_articles_test_clean.csv")
df_test["full_text"] = (df_test["abstract"].fillna("") + " " + df_test["body"].fillna(""))

X_test = pad_sequences(tokenizer.texts_to_sequences(df_test["full_text"].tolist()), maxlen=max_len)
pred_probs = model.predict(X_test)
pred_labels = le.inverse_transform(np.argmax(pred_probs, axis=1))

# ==== 7. Create Submission ====
submission = pd.DataFrame({
    "row_id": range(1, len(df_test)+1),
    "article_id": df_test["article_id"],
    "dataset_id": "unknown",
    "type": pred_labels
})

submission.to_csv("submission_cnn.csv", index=False)
print("\n✅ File submission_cnn.csv berhasil dibuat!")
print(submission.head())


Trial 5 Complete [00h 00m 13s]
val_accuracy: 0.7135922312736511

Best val_accuracy So Far: 0.7281553149223328
Total elapsed time: 00h 01m 52s
Best Hyperparameters:
Embedding Dim: 300
Filters: 256
Kernel Size: 5
Dropout: 0.3
Epoch 1/10
26/26 [==============================] - 9s 325ms/step - loss: 0.8901 - accuracy: 0.5061 - val_loss: 0.7404 - val_accuracy: 0.6845
Epoch 2/10
26/26 [==============================] - 5s 185ms/step - loss: 0.6229 - accuracy: 0.7202 - val_loss: 0.6163 - val_accuracy: 0.6893
Epoch 3/10
26/26 [==============================] - 4s 152ms/step - loss: 0.5059 - accuracy: 0.7676 - val_loss: 0.5736 - val_accuracy: 0.7184
Epoch 4/10
26/26 [==============================] - 4s 141ms/step - loss: 0.4563 - accuracy: 0.7701 - val_loss: 0.5495 - val_accuracy: 0.7282
Epoch 5/10
26/26 [==============================] - 4s 156ms/step - loss: 0.4342 - accuracy: 0.7713 - val_loss: 0.5445 - val_accuracy: 0.7136
Epoch 6/10
26/26 [==============================] - 4s 146ms/step 

In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout
from transformers import AutoTokenizer, AutoModel

# ======================================
# 1. Load Dataset
# ======================================
train_text = pd.read_csv("parsed_articles_train_clean.csv")
test_text = pd.read_csv("parsed_articles_test_clean.csv")
train_labels = pd.read_csv("train_labels.csv")

# Gabungkan teks dengan label
df = train_labels.merge(train_text, on="article_id")
df.fillna("unknown", inplace=True)

# Mapping label ke angka
label_map = {"Primary": 0, "Secondary": 1, "Missing": 2}
df["label"] = df["type"].map(label_map)
df = df.dropna(subset=["label"])

# ======================================
# 2. Train-Validation Split
# ======================================
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
print(f"Train: {train_df.shape}, Val: {val_df.shape}")

# ======================================
# 3. BERT Tokenizer & Embedding
# ======================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased").to(device)

MAX_LEN = 64  # gunakan max_len yg lebih besar utk hasil lebih stabil

def get_bert_embeddings(text_list):
    embeddings = []
    batch_size = 8
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LEN)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = bert_model(**inputs)
        batch_embeddings = outputs.last_hidden_state.cpu().numpy()
        embeddings.append(batch_embeddings)
    return np.concatenate(embeddings, axis=0)

# Gabungkan abstract + body
for df_ in [train_df, val_df, test_text]:
    df_["full_text"] = df_["abstract"].astype(str) + " " + df_["body"].astype(str)

train_texts = train_df["full_text"].tolist()
val_texts = val_df["full_text"].tolist()
test_texts = test_text["full_text"].tolist()

# Proses BERT embedding
X_train = get_bert_embeddings(train_texts)
X_val = get_bert_embeddings(val_texts)
X_test = get_bert_embeddings(test_texts)

y_train = to_categorical(train_df["label"].values, num_classes=3)
y_val = to_categorical(val_df["label"].values, num_classes=3)

print(f"BERT embeddings shape (train): {X_train.shape}")

# ======================================
# 4. Build CNN-BiLSTM Model
# ======================================
model = Sequential([
    Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(MAX_LEN, 768)),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ======================================
# 5. Train the Model
# ======================================
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=4)

# ======================================
# 6. Predict & Submission
# ======================================
preds = model.predict(X_test)
pred_labels = np.argmax(preds, axis=1)

reverse_map = {v: k for k, v in label_map.items()}
pred_types = [reverse_map[i] for i in pred_labels]

submission = test_text[["article_id"]].copy()
submission.insert(0, "row_id", range(1, len(submission) + 1))
submission["type"] = pred_types

submission.to_csv("submission_cnn_bilstm_bert.csv", index=False)
print("✅ File submission_cnn_bilstm_bert.csv berhasil dibuat!")
print(submission.head())


Train: (652, 6), Val: (163, 6)
BERT embeddings shape (train): (652, 64, 768)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 62, 128)           295040    
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 31, 128)          0         
 1D)                                                             
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                           